<a href="https://colab.research.google.com/github/konkuk-gaegul/3rd-Team-Project/blob/main/Modeling/TransferLearning_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 리트리버 학습
- 11개 이미지 세트 중, 가장 성능이 좋은 set 5를 대표로 업로드
- batch_size = 9, epoch = 50인 경우 가장 좋은 성능을 보임
- Loss : 0.1917, Acc : 94.5%

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

import os, shutil
import matplotlib as plt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [ ]:
base_dir = '/home/tutor/안재형/dog_pic/리트리버/set_5'

# # 훈련셋, 검증셋, 테스트셋을 미리 분할하기 위한 폴더 경로
train_path = os.path.join(base_dir, 'train')
os.mkdir( train_path)

test_path = os.path.join(base_dir, 'test')
os.mkdir( test_path)

# # train 정상 폴더
train_nor_path = os.path.join(train_path, 'nor')
os.mkdir( train_nor_path)

# # train 비만 폴더
train_fat_path = os.path.join(train_path, 'fat')
os.mkdir( train_fat_path)

# # test 정상 폴더
test_nor_path = os.path.join(test_path, 'nor')
os.mkdir( test_nor_path)

# # test 비만 폴더
test_fat_path = os.path.join(test_path, 'fat')
os.mkdir( test_fat_path)

In [ ]:
base_dir = '/home/tutor/안재형/dog_pic/리트리버/set_5'

nor_path = os.path.join(base_dir, '전처리_정상')
fat_path = os.path.join(base_dir, '전처리_비만')

nor_list = os.listdir(nor_path)
fat_list = os.listdir(fat_path)

print(len(nor_list))
print(len(fat_list))

577
408


In [ ]:
# train : 75%, test : 25%
# 정상 이미지 복사
for i in range(len(nor_list)):
    src_path = os.path.join(nor_path, nor_list[i])
    
    if i < len(nor_list)*0.75:
        dst_path = os.path.join(train_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_nor_path, f'nor_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
                                
# 비만 이미지 복사
for i in range(len(fat_list)):
    src_path = os.path.join(fat_path, fat_list[i])
    
    if i < len(fat_list)*0.75:
        dst_path = os.path.join(train_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )
    else :
        dst_path = os.path.join(test_fat_path, f'fat_{i}.jpg')
        shutil.copyfile( src_path, dst_path )

In [ ]:
print( len( os.listdir(train_nor_path) ) )
print( len( os.listdir(train_fat_path) ) )
print( len( os.listdir(test_nor_path) ) )
print( len( os.listdir(test_fat_path) ) )

433
306
144
102


# 학습 1
- pytorch로 학습 수행
- 다양한 batch_size, epoch, 이미지 전처리 등으로 최적의 성능을 뽑아낸다.
- 리트리버 set_5로 진행
    - 선명한 이미지의 대비를 높이기 위해 yuv적용 (RGB로 바꾸지 않음)
- batch size를 5부터 20까지 실행
- 이미지 전처리 수행

In [ ]:
# Loss_Accuracy에 각 배치 사이즈마다의 성능을 넣어둔다
Loss_Accuracy = []

In [ ]:
for batch in range(5, 21):
    # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
    # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
    transforms_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
    ])

    transforms_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    data_dir = '/home/tutor/안재형/dog_pic/리트리버/set_5'
    train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
    test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

    train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=True, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch, shuffle=True, num_workers=2)

    print('학습 데이터셋 크기:', len(train_datasets))
    print('테스트 데이터셋 크기:', len(test_datasets))

    class_names = train_datasets.classes
    print('클래스:', class_names)
    
    #--------------------------------------------#
    
    model = models.resnet34(pretrained=True)
    num_features = model.fc.in_features
    # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
    model.fc = nn.Linear(num_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    #--------------------------------------------#
    print(f'-------------현재 batch_size는 {batch}-------------')
    num_epochs = 50
    model.train()
    start_time = time.time()

    # 전체 반복(epoch) 수 만큼 반복하며
    for epoch in range(num_epochs):
        running_loss = 0.
        running_corrects = 0

        # 배치 단위로 학습 데이터 불러오기
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 모델에 입력(forward)하고 결과 계산
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_datasets)
        epoch_acc = running_corrects / len(train_datasets) * 100.

        # 학습 과정 중에 결과 출력
        print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
        
    #--------------------------------------------#
        
    model.eval()
    start_time = time.time()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
            print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
#             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

        epoch_loss = running_loss / len(test_datasets)
        epoch_acc = running_corrects / len(test_datasets) * 100.
        print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))
        
        Loss_Accuracy.append([batch, epoch_loss, epoch_acc])
    # 리트리버 set5의 Batch size가 {batch}인 학습 1의 경우를 의미
    torch.save(model, f'ret_set5_B_{batch}_1.pth')
print( Loss_Accuracy )

학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 batch_size는 5-------------
#0 Loss: 0.7102 Acc: 67.1177% Time: 5.9539s
#1 Loss: 0.5850 Acc: 76.3194% Time: 11.7813s
#2 Loss: 0.4834 Acc: 80.3789% Time: 17.6404s
#3 Loss: 0.3712 Acc: 85.5210% Time: 23.4860s
#4 Loss: 0.3907 Acc: 86.0622% Time: 29.3711s
#5 Loss: 0.3125 Acc: 87.5507% Time: 35.2619s
#6 Loss: 0.4026 Acc: 86.8742% Time: 41.1269s
#7 Loss: 0.2969 Acc: 90.3924% Time: 46.9996s
#8 Loss: 0.2608 Acc: 89.9865% Time: 52.8669s
#9 Loss: 0.2502 Acc: 92.0162% Time: 58.7766s
#10 Loss: 0.2355 Acc: 92.0162% Time: 64.7088s
#11 Loss: 0.2358 Acc: 91.8809% Time: 70.6648s
#12 Loss: 0.1197 Acc: 95.5345% Time: 76.6281s
#13 Loss: 0.1336 Acc: 96.0758% Time: 82.5564s
#14 Loss: 0.2090 Acc: 92.6928% Time: 88.5012s
#15 Loss: 0.1444 Acc: 95.3992% Time: 94.4755s
#16 Loss: 0.1251 Acc: 96.3464% Time: 100.5021s
#17 Loss: 0.1321 Acc: 96.2111% Time: 106.5047s
#18 Loss: 0.0824 Acc: 97.5643% Time: 112.5735s
#19 Loss: 0.0978 Acc: 96.8877% Time: 1

#18 Loss: 0.0827 Acc: 97.1583% Time: 108.9199s
#19 Loss: 0.0648 Acc: 98.5115% Time: 114.6250s
#20 Loss: 0.1024 Acc: 97.1583% Time: 120.4065s
#21 Loss: 0.0434 Acc: 98.3762% Time: 126.1171s
#22 Loss: 0.0537 Acc: 98.3762% Time: 131.8365s
#23 Loss: 0.0202 Acc: 99.3234% Time: 137.5490s
#24 Loss: 0.0262 Acc: 99.1881% Time: 143.2887s
#25 Loss: 0.0373 Acc: 98.5115% Time: 149.0153s
#26 Loss: 0.0662 Acc: 98.3762% Time: 154.7431s
#27 Loss: 0.0218 Acc: 99.4587% Time: 160.4953s
#28 Loss: 0.0358 Acc: 98.6468% Time: 166.1987s
#29 Loss: 0.0524 Acc: 98.3762% Time: 171.9781s
#30 Loss: 0.0229 Acc: 99.4587% Time: 177.7923s
#31 Loss: 0.0260 Acc: 98.7821% Time: 183.4734s
#32 Loss: 0.0295 Acc: 99.1881% Time: 189.1713s
#33 Loss: 0.0218 Acc: 99.1881% Time: 194.9025s
#34 Loss: 0.0255 Acc: 99.3234% Time: 200.6163s
#35 Loss: 0.0323 Acc: 98.7821% Time: 206.3362s
#36 Loss: 0.0112 Acc: 99.7294% Time: 212.0471s
#37 Loss: 0.0118 Acc: 99.5940% Time: 217.7400s
#38 Loss: 0.0179 Acc: 99.5940% Time: 223.4424s
#39 Loss: 0.0

[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[Test Phase] Loss: 0.1917 Acc: 94.3089% Time: 1.0101s
학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 batch_size는 10-------------
#0 Loss: 0.5603 Acc: 70.7713% Time: 5.2250s
#1 Loss: 0.3533 Acc: 85.5210% Time: 10.5133s
#2 Loss: 0.2688 Acc: 88.6333% Time: 15.7509s
#3 Loss: 0.2361 Acc: 90.7984% Time: 20.8421s
#4 Loss: 0.1861 Acc:

#27 Loss: 0.0315 Acc: 98.9175% Time: 144.4668s
#28 Loss: 0.0397 Acc: 98.5115% Time: 149.7691s
#29 Loss: 0.0170 Acc: 99.3234% Time: 154.9058s
#30 Loss: 0.0109 Acc: 99.5940% Time: 160.0063s
#31 Loss: 0.0168 Acc: 99.4587% Time: 165.1495s
#32 Loss: 0.0127 Acc: 99.5940% Time: 170.2307s
#33 Loss: 0.0196 Acc: 99.3234% Time: 175.3288s
#34 Loss: 0.0232 Acc: 98.7821% Time: 180.4180s
#35 Loss: 0.0280 Acc: 99.3234% Time: 185.6792s
#36 Loss: 0.0104 Acc: 99.7294% Time: 190.8725s
#37 Loss: 0.0264 Acc: 99.3234% Time: 195.9721s
#38 Loss: 0.0188 Acc: 99.3234% Time: 201.1117s
#39 Loss: 0.0179 Acc: 99.0528% Time: 206.2434s
#40 Loss: 0.0213 Acc: 99.0528% Time: 211.3282s
#41 Loss: 0.0174 Acc: 99.5940% Time: 216.4715s
#42 Loss: 0.0090 Acc: 99.4587% Time: 221.7323s
#43 Loss: 0.0192 Acc: 99.1881% Time: 226.9072s
#44 Loss: 0.0210 Acc: 99.3234% Time: 231.9454s
#45 Loss: 0.0118 Acc: 99.4587% Time: 237.0442s
#46 Loss: 0.0254 Acc: 99.1881% Time: 242.0509s
#47 Loss: 0.0265 Acc: 99.0528% Time: 247.2390s
#48 Loss: 0.0

#12 Loss: 0.0526 Acc: 98.2409% Time: 63.3821s
#13 Loss: 0.0436 Acc: 98.5115% Time: 68.2148s
#14 Loss: 0.0305 Acc: 99.0528% Time: 73.0602s
#15 Loss: 0.0253 Acc: 99.1881% Time: 78.2205s
#16 Loss: 0.0367 Acc: 98.6468% Time: 82.9566s
#17 Loss: 0.0842 Acc: 97.5643% Time: 87.8733s
#18 Loss: 0.0929 Acc: 97.0230% Time: 92.9177s
#19 Loss: 0.1103 Acc: 96.0758% Time: 97.9349s
#20 Loss: 0.0541 Acc: 97.8349% Time: 102.7017s
#21 Loss: 0.0538 Acc: 98.2409% Time: 107.5469s
#22 Loss: 0.0381 Acc: 99.1881% Time: 112.4943s
#23 Loss: 0.0534 Acc: 98.3762% Time: 117.4247s
#24 Loss: 0.0506 Acc: 97.9702% Time: 122.4618s
#25 Loss: 0.0386 Acc: 98.3762% Time: 127.2508s
#26 Loss: 0.0314 Acc: 99.1881% Time: 132.4628s
#27 Loss: 0.1242 Acc: 96.7524% Time: 137.5505s
#28 Loss: 0.0443 Acc: 98.7821% Time: 142.3828s
#29 Loss: 0.0256 Acc: 99.1881% Time: 147.3828s
#30 Loss: 0.0284 Acc: 98.9175% Time: 152.1239s
#31 Loss: 0.0165 Acc: 99.4587% Time: 157.0663s
#32 Loss: 0.0207 Acc: 99.3234% Time: 161.9741s
#33 Loss: 0.0125 Acc:

#2 Loss: 0.3172 Acc: 86.0622% Time: 15.6145s
#3 Loss: 0.2332 Acc: 90.3924% Time: 20.8063s
#4 Loss: 0.1794 Acc: 93.6401% Time: 26.1202s
#5 Loss: 0.1670 Acc: 93.6401% Time: 31.3159s
#6 Loss: 0.1213 Acc: 94.9932% Time: 36.6581s
#7 Loss: 0.1164 Acc: 96.2111% Time: 41.8438s
#8 Loss: 0.0951 Acc: 96.0758% Time: 47.0104s
#9 Loss: 0.0824 Acc: 97.0230% Time: 52.1588s
#10 Loss: 0.1033 Acc: 96.3464% Time: 57.3150s
#11 Loss: 0.1636 Acc: 95.6698% Time: 62.5898s
#12 Loss: 0.0644 Acc: 97.0230% Time: 67.7895s
#13 Loss: 0.0565 Acc: 97.5643% Time: 73.2110s
#14 Loss: 0.0538 Acc: 97.9702% Time: 78.3283s
#15 Loss: 0.0577 Acc: 97.6996% Time: 83.7054s
#16 Loss: 0.0573 Acc: 97.4290% Time: 88.9495s
#17 Loss: 0.0570 Acc: 97.6996% Time: 94.0557s
#18 Loss: 0.0595 Acc: 98.2409% Time: 99.2566s
#19 Loss: 0.0124 Acc: 99.7294% Time: 104.3864s
#20 Loss: 0.0418 Acc: 98.5115% Time: 109.6306s
#21 Loss: 0.0389 Acc: 98.3762% Time: 114.9122s
#22 Loss: 0.0448 Acc: 98.1055% Time: 120.0649s
#23 Loss: 0.0736 Acc: 97.6996% Time: 1

[[5, 0.32808656001240405, tensor(91.8699, device='cuda:0')], [6, 0.3622625100592481, tensor(91.8699, device='cuda:0')], [7, 0.35845476735581483, tensor(92.6829, device='cuda:0')], [8, 0.298504257616322, tensor(91.8699, device='cuda:0')], [9, 0.19171841152398506, tensor(94.3089, device='cuda:0')], [10, 0.25149055655167357, tensor(93.4959, device='cuda:0')], [11, 0.36617710030395384, tensor(91.0569, device='cuda:0')], [12, 0.33526861287721593, tensor(91.8699, device='cuda:0')], [13, 0.29604436570165177, tensor(92.2764, device='cuda:0')], [14, 0.32727507164898684, tensor(91.8699, device='cuda:0')], [15, 0.238802542932695, tensor(93.9024, device='cuda:0')], [16, 0.26581978348909113, tensor(95.1219, device='cuda:0')], [17, 0.28264929994759036, tensor(94.7154, device='cuda:0')], [18, 0.35929426152213684, tensor(93.9024, device='cuda:0')], [19, 0.24946958038797276, tensor(94.3089, device='cuda:0')], [20, 0.2959801397717944, tensor(92.6829, device='cuda:0')]]


In [ ]:
for i in range(10):
    print(f'batch가 {Loss_Accuracy[i][0]}인 경우 Loss는 {Loss_Accuracy[i][1]}')

batch가 5인 경우 Loss는 0.32808656001240405
batch가 6인 경우 Loss는 0.3622625100592481
batch가 7인 경우 Loss는 0.35845476735581483
batch가 8인 경우 Loss는 0.298504257616322
batch가 9인 경우 Loss는 0.19171841152398506
batch가 10인 경우 Loss는 0.25149055655167357
batch가 11인 경우 Loss는 0.36617710030395384
batch가 12인 경우 Loss는 0.33526861287721593
batch가 13인 경우 Loss는 0.29604436570165177
batch가 14인 경우 Loss는 0.32727507164898684


- set 3과 비슷하게 대체로 batch_size = 9 전후로 성능이 가장 좋은 모습을 보인다

# 학습 2
- batch_size는 9
- epoch를 30, 50, 75, 100 으로 해보자

In [ ]:
# 빈 문자열에 각 배치 사이즈마다의 성능을 넣어둔다
Loss_Accuracy = []
Epoch = [30, 50, 75, 100]

In [ ]:
for epoch in Epoch:
    # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
    # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
    transforms_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
    ])

    transforms_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    data_dir = '/home/tutor/안재형/dog_pic/리트리버/set_5'
    train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
    test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

    train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=9, shuffle=True, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=9, shuffle=True, num_workers=2)

    print('학습 데이터셋 크기:', len(train_datasets))
    print('테스트 데이터셋 크기:', len(test_datasets))

    class_names = train_datasets.classes
    print('클래스:', class_names)
    
    #--------------------------------------------#
    
    model = models.resnet34(pretrained=True)
    num_features = model.fc.in_features
    # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
    model.fc = nn.Linear(num_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    #--------------------------------------------#
    print(f'-------------현재 학습의 Epoch는 {epoch}-------------')
    num_epochs = epoch
    model.train()
    start_time = time.time()

    # 전체 반복(epoch) 수 만큼 반복하며
    for epoch in range(num_epochs):
        running_loss = 0.
        running_corrects = 0

        # 배치 단위로 학습 데이터 불러오기
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 모델에 입력(forward)하고 결과 계산
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_datasets)
        epoch_acc = running_corrects / len(train_datasets) * 100.

        # 학습 과정 중에 결과 출력
        print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
        
    #--------------------------------------------#
        
    model.eval()
    start_time = time.time()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
            print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
#             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

        epoch_loss = running_loss / len(test_datasets)
        epoch_acc = running_corrects / len(test_datasets) * 100.
        print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))
        
        Loss_Accuracy.append([epoch, epoch_loss, epoch_acc])
    # 리트리버 set5의 Batch size가 {batch}인 학습 1의 경우를 의미
    torch.save(model, f'ret_set5_E_{epoch}_2.pth')
print( Loss_Accuracy )

학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 학습의 Epoch는 30-------------
#0 Loss: 0.6916 Acc: 63.7348% Time: 5.4027s
#1 Loss: 0.4336 Acc: 79.1610% Time: 10.7978s
#2 Loss: 0.3579 Acc: 86.7388% Time: 16.0413s
#3 Loss: 0.2737 Acc: 87.9567% Time: 21.3541s
#4 Loss: 0.2480 Acc: 90.5277% Time: 26.7116s
#5 Loss: 0.1740 Acc: 93.0988% Time: 32.0164s
#6 Loss: 0.1333 Acc: 94.9932% Time: 37.3571s
#7 Loss: 0.1918 Acc: 92.4222% Time: 42.6771s
#8 Loss: 0.1724 Acc: 94.1813% Time: 48.0149s
#9 Loss: 0.1285 Acc: 95.3992% Time: 53.4665s
#10 Loss: 0.1090 Acc: 95.9405% Time: 58.7883s
#11 Loss: 0.0980 Acc: 95.8051% Time: 64.2649s
#12 Loss: 0.1095 Acc: 95.6698% Time: 69.8511s
#13 Loss: 0.1154 Acc: 96.4817% Time: 75.2611s
#14 Loss: 0.1057 Acc: 96.8877% Time: 80.7157s
#15 Loss: 0.0772 Acc: 97.1583% Time: 86.1163s
#16 Loss: 0.0808 Acc: 96.8877% Time: 91.4550s
#17 Loss: 0.0843 Acc: 96.6171% Time: 96.8833s
#18 Loss: 0.0849 Acc: 97.2936% Time: 102.5261s
#19 Loss: 0.1849 Acc: 94.5873% Time: 108

#57 Loss: 0.0111 Acc: 99.4587% Time: 323.2566s
#58 Loss: 0.0200 Acc: 99.5940% Time: 328.7159s
#59 Loss: 0.0111 Acc: 99.5940% Time: 334.2847s
#60 Loss: 0.0773 Acc: 98.1055% Time: 339.9066s
#61 Loss: 0.0122 Acc: 99.7294% Time: 345.3728s
#62 Loss: 0.0352 Acc: 99.5940% Time: 350.9596s
#63 Loss: 0.0277 Acc: 99.0528% Time: 356.5003s
#64 Loss: 0.0072 Acc: 100.0000% Time: 362.2452s
#65 Loss: 0.0085 Acc: 99.7294% Time: 367.7360s
#66 Loss: 0.0296 Acc: 98.9175% Time: 373.2787s
#67 Loss: 0.0221 Acc: 99.1881% Time: 378.7788s
#68 Loss: 0.0449 Acc: 98.7821% Time: 384.2784s
#69 Loss: 0.0385 Acc: 98.7821% Time: 389.8234s
#70 Loss: 0.0069 Acc: 99.7294% Time: 395.2860s
#71 Loss: 0.0072 Acc: 99.7294% Time: 400.9791s
#72 Loss: 0.0073 Acc: 99.8647% Time: 406.4557s
#73 Loss: 0.0098 Acc: 99.4587% Time: 411.9920s
#74 Loss: 0.0290 Acc: 99.0528% Time: 417.6091s
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fa

In [ ]:
for i in range(4):
    print(f'epoch가 {Loss_Accuracy[i][0]}인 경우 Loss는 {Loss_Accuracy[i][1]}')

epoch가 29인 경우 Loss는 0.40108131255342083
epoch가 49인 경우 Loss는 0.32244573027554263
epoch가 74인 경우 Loss는 0.6445640587127173
epoch가 99인 경우 Loss는 0.32355097715228065


- epoch가 50, 100인 경우 가장 좋은 성능을 낸다.
- 매번 같은 조건에서 학습을 해도 다른 성능을 내기 때문에 최적의 설정을 수십 번 반복해서
- 가장 좋은 성능을 저장하는 방법도 좋아보인다.

# 학습 3
- batch_size = 9
- epoch = 50
- 위의 두 설정으로 100번 반복 학습
- 학습 후에 제일 좋은 모델 1개 선정

In [ ]:
!pwd

/home/tutor/안재형/학습 모델/리트리버/set_5


In [ ]:
%cd /home/tutor/안재형/saved_files/리트리버/iteration_set_5

/home/tutor/안재형/saved_files/리트리버/iteration_set_5


In [ ]:
i = 1

for _ in range(150):
    # 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
    # 이미지의 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 일부 변경
    transforms_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
    ])

    transforms_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    data_dir = '/home/tutor/안재형/dog_pic/리트리버/set_5'
    train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
    test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

    train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=9, shuffle=True, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=9, shuffle=True, num_workers=2)

    print('학습 데이터셋 크기:', len(train_datasets))
    print('테스트 데이터셋 크기:', len(test_datasets))

    class_names = train_datasets.classes
    print('클래스:', class_names)
    
    #--------------------------------------------#
    
    model = models.resnet34(pretrained=True)
    num_features = model.fc.in_features
    # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 2개로 교체하여 마지막 레이어 다시 학습
    model.fc = nn.Linear(num_features, 2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    #--------------------------------------------#
    print(f'-------------현재 {i}번째 학습 중-------------')
    num_epochs = 50
    model.train()
    start_time = time.time()

    # 전체 반복(epoch) 수 만큼 반복하며
    for epoch in range(num_epochs):
        running_loss = 0.
        running_corrects = 0

        # 배치 단위로 학습 데이터 불러오기
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 모델에 입력(forward)하고 결과 계산
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_datasets)
        epoch_acc = running_corrects / len(train_datasets) * 100.

        # 학습 과정 중에 결과 출력
        print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
        
    #--------------------------------------------#
        
    model.eval()
    start_time = time.time()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
            print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
#             imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

        epoch_loss = running_loss / len(test_datasets)
        epoch_acc = running_corrects / len(test_datasets) * 100.
        print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))
        
        # 우선 모든 모델 성능 저장
        Loss_Accuracy.append([i, epoch_loss, epoch_acc])
    
    # 150개 모든 모델을 저장하지 않고 가장 좋은 성능의 모델을 저장
    if i == 1:
        Best = [[i, epoch_loss]]
        # 리트리버 set5/batch_size = 9의 {i}번째, 학습 3인 경우를 의미
        torch.save(model, f'ret_set5_B9_{i}_3.pth')
    
    elif epoch_loss < Best[0][1]:
        # Best 모델보다 Loss가 개선되면 저장
        Best = [[i, epoch_loss]]
        # 가장 마지막으로 저장된 모델이 제일 좋은 성능
        torch.save(model, f'ret_set5_B9_{i}_3.pth')
    
    i += 1

학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 1번째 학습 중-------------
#0 Loss: 0.5479 Acc: 71.1773% Time: 5.3893s
#1 Loss: 0.4166 Acc: 81.8674% Time: 10.6963s
#2 Loss: 0.3860 Acc: 85.2503% Time: 15.9802s
#3 Loss: 0.2259 Acc: 90.5277% Time: 21.2110s
#4 Loss: 0.2343 Acc: 91.0690% Time: 26.4548s
#5 Loss: 0.2452 Acc: 91.4750% Time: 31.8305s
#6 Loss: 0.2040 Acc: 93.5047% Time: 37.2528s
#7 Loss: 0.1052 Acc: 95.3992% Time: 42.6371s
#8 Loss: 0.1254 Acc: 94.8579% Time: 47.9549s
#9 Loss: 0.1691 Acc: 93.6401% Time: 53.3187s
#10 Loss: 0.1479 Acc: 95.2639% Time: 58.5607s
#11 Loss: 0.0981 Acc: 95.9405% Time: 63.8491s
#12 Loss: 0.1207 Acc: 95.9405% Time: 69.2884s
#13 Loss: 0.1164 Acc: 95.6698% Time: 74.5840s
#14 Loss: 0.1386 Acc: 96.2111% Time: 79.7907s
#15 Loss: 0.0977 Acc: 97.1583% Time: 85.1183s
#16 Loss: 0.1210 Acc: 96.7524% Time: 90.4420s
#17 Loss: 0.1091 Acc: 96.0758% Time: 95.9000s
#18 Loss: 0.0741 Acc: 97.2936% Time: 101.2125s
#19 Loss: 0.0562 Acc: 98.3762% Time: 106.5907

#38 Loss: 0.0396 Acc: 98.1055% Time: 217.5970s
#39 Loss: 0.0217 Acc: 99.0528% Time: 223.0769s
#40 Loss: 0.0374 Acc: 98.9175% Time: 228.8154s
#41 Loss: 0.0247 Acc: 99.3234% Time: 234.3395s
#42 Loss: 0.0201 Acc: 99.3234% Time: 239.8559s
#43 Loss: 0.0206 Acc: 98.6468% Time: 245.3336s
#44 Loss: 0.0190 Acc: 99.4587% Time: 250.8266s
#45 Loss: 0.0395 Acc: 98.7821% Time: 256.3763s
#46 Loss: 0.0600 Acc: 97.9702% Time: 262.0475s
#47 Loss: 0.0315 Acc: 99.0528% Time: 267.5301s
#48 Loss: 0.0175 Acc: 99.5940% Time: 273.0252s
#49 Loss: 0.0212 Acc: 99.0528% Time: 278.5887s
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답:

#9 Loss: 0.1093 Acc: 96.2111% Time: 55.6081s
#10 Loss: 0.1343 Acc: 94.5873% Time: 61.3535s
#11 Loss: 0.0862 Acc: 96.7524% Time: 67.0752s
#12 Loss: 0.1480 Acc: 94.4520% Time: 72.5850s
#13 Loss: 0.0873 Acc: 97.1583% Time: 78.1304s
#14 Loss: 0.1035 Acc: 95.3992% Time: 83.8197s
#15 Loss: 0.0977 Acc: 96.0758% Time: 89.4134s
#16 Loss: 0.1172 Acc: 96.4817% Time: 95.0466s
#17 Loss: 0.1388 Acc: 95.6698% Time: 100.5249s
#18 Loss: 0.0447 Acc: 98.7821% Time: 106.0714s
#19 Loss: 0.1064 Acc: 96.4817% Time: 111.7968s
#20 Loss: 0.0783 Acc: 97.4290% Time: 117.4636s
#21 Loss: 0.0320 Acc: 99.1881% Time: 122.9351s
#22 Loss: 0.0179 Acc: 99.3234% Time: 128.6947s
#23 Loss: 0.0258 Acc: 98.9175% Time: 134.2974s
#24 Loss: 0.0416 Acc: 98.3762% Time: 139.9935s
#25 Loss: 0.0263 Acc: 98.7821% Time: 145.5584s
#26 Loss: 0.0214 Acc: 99.1881% Time: 151.0448s
#27 Loss: 0.0191 Acc: 99.0528% Time: 156.4014s
#28 Loss: 0.0431 Acc: 98.5115% Time: 161.7306s
#29 Loss: 0.0652 Acc: 98.5115% Time: 167.2159s
#30 Loss: 0.0766 Acc: 

#48 Loss: 0.0624 Acc: 97.1583% Time: 265.5611s
#49 Loss: 0.0583 Acc: 98.3762% Time: 271.1881s
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[Test Phase] Loss: 0.3017 Acc: 91.8699% Time: 0.8988s
학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 9번째 학습 중-------------
#0 Loss: 0.6176 Acc: 68.2003% Tim

#19 Loss: 0.0969 Acc: 96.8877% Time: 111.7320s
#20 Loss: 0.0713 Acc: 97.0230% Time: 117.2039s
#21 Loss: 0.0991 Acc: 96.4817% Time: 122.6658s
#22 Loss: 0.0614 Acc: 97.4290% Time: 128.1962s
#23 Loss: 0.0304 Acc: 98.7821% Time: 133.6388s
#24 Loss: 0.0586 Acc: 97.6996% Time: 139.2594s
#25 Loss: 0.0306 Acc: 98.9175% Time: 144.7079s
#26 Loss: 0.0515 Acc: 97.8349% Time: 150.2471s
#27 Loss: 0.0532 Acc: 98.5115% Time: 155.8079s
#28 Loss: 0.0375 Acc: 98.7821% Time: 161.2648s
#29 Loss: 0.0619 Acc: 98.1055% Time: 166.8558s
#30 Loss: 0.0824 Acc: 97.1583% Time: 172.4099s
#31 Loss: 0.0376 Acc: 98.6468% Time: 178.0031s
#32 Loss: 0.0230 Acc: 99.0528% Time: 183.5288s
#33 Loss: 0.0467 Acc: 98.3762% Time: 188.9748s
#34 Loss: 0.0341 Acc: 99.3234% Time: 194.5704s
#35 Loss: 0.0403 Acc: 98.5115% Time: 200.2756s
#36 Loss: 0.0378 Acc: 98.6468% Time: 206.0180s
#37 Loss: 0.0590 Acc: 97.6996% Time: 211.5041s
#38 Loss: 0.0405 Acc: 98.5115% Time: 217.1439s
#39 Loss: 0.0269 Acc: 98.9175% Time: 222.6318s
#40 Loss: 0.0

...

[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[Test Phase] Loss: 0.3189 Acc: 92.6829% Time: 0.8897s
학습 데이터셋 크기: 739
테스트 데이터셋 크기: 246
클래스: ['fat', 'nor']
-------------현재 140번째 학습 중-------------
#0 Loss: 0.6558 Acc: 66.4411% Time: 5.4536s
#1 Loss: 0.3690 Acc: 85.3857% Time: 10.7283s
#2 Loss: 0.3129 Acc: 87.0095% Time: 15.9995s
#3 Loss: 0.2647 Acc: 88.9039% Time: 21.9029s
#4 Loss: 0.2937 Acc: 89.0392% Time: 31.9274s
#5 Loss: 0.1750 Acc: 93.7754% Time: 41.0312s
#6 Loss: 0.2028 Acc: 92.4222% Time: 46.3294s
#7 Loss: 0.2679 Acc: 91.3396% Time: 51.7805s
#8 Loss: 0.1077 Acc: 96.2111% Time: 57.0375s
#9 Loss: 0.1961 Acc: 93.3694% Time: 62.3035s
#10 Loss: 0.1247 Acc: 95.5345% Time: 67.9571s
#11 Loss: 0.1655 Acc: 94.8579% Time: 77.8447s
#12 Loss: 0.0860 Acc: 96.8877% Time: 87.6740s
#13 Loss: 0.0987 Acc: 96.8877% Time: 93.3243s
#14 Loss: 0.0939 Acc: 96.2111% Time: 98.5172s
#15 Loss: 0.0600 Acc: 97.4290% Time: 103.8796s
#16 Loss: 0.0785 Acc: 97.1583% Time: 113.0693s
#17 Loss: 0.0860 Acc: 97.158

#35 Loss: 0.0233 Acc: 99.3234% Time: 199.1928s
#36 Loss: 0.0311 Acc: 98.7821% Time: 204.8136s
#37 Loss: 0.0645 Acc: 97.2936% Time: 210.4934s
#38 Loss: 0.0252 Acc: 99.0528% Time: 216.0325s
#39 Loss: 0.0335 Acc: 98.9175% Time: 221.7410s
#40 Loss: 0.0541 Acc: 98.6468% Time: 227.2276s
#41 Loss: 0.0550 Acc: 98.2409% Time: 232.8496s
#42 Loss: 0.0288 Acc: 99.0528% Time: 238.3500s
#43 Loss: 0.0231 Acc: 99.1881% Time: 243.9541s
#44 Loss: 0.0328 Acc: 98.5115% Time: 249.4711s
#45 Loss: 0.0205 Acc: 99.4587% Time: 255.1933s
#46 Loss: 0.0363 Acc: 99.0528% Time: 260.6610s
#47 Loss: 0.0039 Acc: 100.0000% Time: 266.4268s
#48 Loss: 0.0307 Acc: 99.0528% Time: 271.9555s
#49 Loss: 0.0177 Acc: 99.1881% Time: 277.6566s
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: 

#5 Loss: 0.3354 Acc: 88.2273% Time: 33.1188s
#6 Loss: 0.1728 Acc: 93.7754% Time: 38.6363s
#7 Loss: 0.1694 Acc: 94.5873% Time: 44.1930s
#8 Loss: 0.0909 Acc: 96.4817% Time: 49.8319s
#9 Loss: 0.0998 Acc: 96.4817% Time: 55.2993s
#10 Loss: 0.1535 Acc: 94.5873% Time: 60.8082s
#11 Loss: 0.1434 Acc: 93.6401% Time: 66.4011s
#12 Loss: 0.0988 Acc: 96.2111% Time: 71.8834s
#13 Loss: 0.1289 Acc: 95.1285% Time: 77.4448s
#14 Loss: 0.1140 Acc: 96.3464% Time: 82.8908s
#15 Loss: 0.0833 Acc: 96.8877% Time: 88.3867s
#16 Loss: 0.0852 Acc: 97.2936% Time: 94.0715s
#17 Loss: 0.0594 Acc: 97.2936% Time: 99.5755s
#18 Loss: 0.0556 Acc: 97.8349% Time: 105.1098s
#19 Loss: 0.1042 Acc: 96.8877% Time: 110.6316s
#20 Loss: 0.1381 Acc: 95.8051% Time: 116.1757s
#21 Loss: 0.1170 Acc: 95.9405% Time: 121.6776s
#22 Loss: 0.0932 Acc: 96.6171% Time: 127.2128s
#23 Loss: 0.0407 Acc: 98.7821% Time: 132.8397s
#24 Loss: 0.0202 Acc: 99.4587% Time: 138.3131s
#25 Loss: 0.0743 Acc: 97.4290% Time: 143.8166s
#26 Loss: 0.0865 Acc: 97.0230% 

#44 Loss: 0.0224 Acc: 99.3234% Time: 250.3360s
#45 Loss: 0.0351 Acc: 98.5115% Time: 255.9045s
#46 Loss: 0.0587 Acc: 98.6468% Time: 261.5806s
#47 Loss: 0.0109 Acc: 99.4587% Time: 267.0973s
#48 Loss: 0.0218 Acc: 99.0528% Time: 272.6122s
#49 Loss: 0.0659 Acc: 97.9702% Time: 278.2097s
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: fat] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: fat)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제 정답: nor)
[예측 결과: nor] (실제

#15 Loss: 0.0717 Acc: 97.5643% Time: 85.5227s
#16 Loss: 0.1073 Acc: 96.3464% Time: 90.7782s
#17 Loss: 0.0632 Acc: 97.8349% Time: 96.2206s
#18 Loss: 0.1336 Acc: 95.9405% Time: 101.5501s
#19 Loss: 0.1129 Acc: 96.6171% Time: 106.8537s
#20 Loss: 0.0854 Acc: 97.0230% Time: 112.3097s
#21 Loss: 0.0699 Acc: 97.5643% Time: 117.7231s
#22 Loss: 0.1122 Acc: 97.1583% Time: 123.1951s
#23 Loss: 0.0960 Acc: 96.7524% Time: 128.7843s
#24 Loss: 0.0748 Acc: 97.6996% Time: 134.1550s
#25 Loss: 0.0703 Acc: 96.8877% Time: 139.6059s
#26 Loss: 0.0669 Acc: 97.2936% Time: 145.0921s
#27 Loss: 0.0433 Acc: 98.3762% Time: 150.5851s
#28 Loss: 0.0467 Acc: 98.2409% Time: 156.0645s
#29 Loss: 0.0517 Acc: 97.5643% Time: 161.8049s
#30 Loss: 0.0776 Acc: 98.5115% Time: 167.3076s
#31 Loss: 0.0304 Acc: 99.0528% Time: 172.8221s
#32 Loss: 0.0520 Acc: 98.9175% Time: 178.4284s
#33 Loss: 0.0390 Acc: 98.9175% Time: 183.8577s
#34 Loss: 0.0221 Acc: 99.0528% Time: 189.3334s
#35 Loss: 0.0211 Acc: 99.1881% Time: 194.8206s
#36 Loss: 0.0110

In [ ]:
Best

[[38, 0.20017548529473814]]

- 100중 38번 째 에서 가장 좋은 성능을 보인다.

# 리트리버 한계점
- 리트리버의 털은 수북한 특징을 띈다.
- 따라서 객체 윤곽과 배경의 구분이 명확하지 않은 한계점이 존재한다.

# 리트리버 개선사항
- 객체 윤곽과 배경을 뚜렷하게 구분할 수 있게 이미지의 밝기, 대비 등 전처리를 다양하게 적용해보자